In [1]:
from pathlib import Path

from util import p_

import automech


class Eq:
    allyl = "C5H8(522) + OH(4) = C5H7(500) + H2O(5)"
    alkyl = "C5H8(522) + OH(4) = C5H7(504) + H2O(5)"
    vinyl = "C5H8(522) + OH(4) = C5H7(1202) + H2O(5)"

tag = "A_rh-oh_1e-1_v0"
root_path = Path("..") / Path("..")

In [2]:
par_mech = automech.io.read(p_.parent_mechanism("json", path=p_.data(root_path)))
cal_mech = automech.io.read(
    p_.calculated_mechanism(tag, "json", path=p_.data(root_path))
)

In [3]:
par_allyl_rate = automech.reaction_rate_objects(par_mech, Eq.allyl)

In [4]:
cal_allyl_rate = automech.reaction_rate_objects(cal_mech, Eq.allyl)
cal_allyl_rate

[Reaction(reactants=['C5H8(522)', 'OH(4)'], products=['C5H7(500)', 'H2O(5)'], reversible=True, rate=ArrheniusRateFit(order=2, efficiencies={}, A=2675000.0, b=2.041, E=-2679.0, type='arrhenius'))]